In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/petfinder-densenet/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5
/kaggle/input/petfinder-densenet/densenet121weights/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5
/kaggle/input/pisangtest/pisangtest/kepoks (80).jpg
/kaggle/input/pisangtest/pisangtest/ambon99.jpg
/kaggle/input/pisangtest/pisangtest/kepoks (63).jpg
/kaggle/input/pisangtest/pisangtest/ambon3.jpg
/kaggle/input/pisangtest/pisangtest/ambon6.jpg
/kaggle/input/pisangtest/pisangtest/ambon79.jpg
/kaggle/input/pisangtest/pisangtest/ambon7.jpg
/kaggle/input/pisangtest/pisangtest/ambon80.jpg
/kaggle/input/pisangtest/pisangtest/ambon5.jpg
/kaggle/input/pisangtest/pisangtest/kepoks (60).jpg
/kaggle/input/pisangtest/pisangtest/kepoks (70).jpg
/kaggle/input/pisangtest/pisangtest/kepoks (100).jpg
/kaggle/input/pisangtest/pisangtest/ambon2.jpg
/kaggle/input/pisangtest/pisangtest/ambon1.jpg
/kaggle/input/pisangtest/pisangtest/ambon95.jpg
/kaggle/input/pisangtest/pisangtest/kepoks (86).jpg
/kaggle/input/pi

Pendahuluan
Keras memiliki banyak model deep learning untuk klasifikasi gambar yang siap pakai. Dengan mengatur parameter weights='imagenet', weight hasil training berhari-hari pada jutaan gambar dari dataset ImageNet otomatis didownload. Tanpa kita training di PC sendiri, model dapat kita masukkan gambar sembarang untuk diklasifikasi.

Jumlah kelas dataset ImageNet sangat banyak, ada 1000. Perhatikan sampel berikut.

ImageNet classes.txt
... head cabbage broccoli cauliflower zucchini, courgette spaghetti squash acorn squash butternut squash cucumber, cuke artichoke, globe artichoke bell pepper cardoon mushroom Granny Smith strawberry orange lemon fig pineapple, ananas banana jackfruit, jak, jack custard apple pomegranate ...

Model mampu membedakan berbagai sayur dan buah. Akan tetapi, model tidak cukup spesifik untuk membedakan varian dari setiap sayur atau buah dalam perbendaharaannya. Model akan memprediksi, misalnya, 'orange' ketika dimasukkan gambar jeruk mandarin maupun jeruk sunkist. Pada artikel ini kita akan membuat model yang khusus membedakan antara pisang tanduk dan pisang kepok. Kita akan menggunakan salah satu model yang disediakan Keras, DenseNet121.

DenseNet
DenseNet models {121, 169, 201}, with weights pre-trained on ImageNet.

This model and can be built both with 'channels_first' data format (channels, height, width) or 'channels_last' data format (height, width, channels).

The default input size for this model is 224x224.

keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
Arguments
blocks: numbers of building blocks for the four dense layers.
include_top: whether to include the fully-connected layer at the top of the network.
weights: one of None (random initialization), 'imagenet' (pre-training on ImageNet), or the path to the weights file to be loaded.
input_tensor: optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model.
input_shape: optional shape tuple, only to be specified if include_top is False (otherwise the input shape has to be (224, 224, 3) (with 'channels_last' data format) or (3, 224, 224) (with 'channels_first' data format). It should have exactly 3 inputs channels, and width and height should be no smaller than 32. E.g. (200, 200, 3) would be one valid value.
pooling: optional pooling mode for feature extraction when include_top is False. None means that the output of the model will be the 4D tensor output of the last convolutional layer. avg means that global average pooling will be applied to the output of the last convolutional layer, and thus the output of the model will be a 2D tensor. max means that global max pooling will be applied.
classes: optional number of classes to classify images into, only to be specified if include_top is True, and if no weights argument is specified.

In [2]:
from keras.applications import densenet
from keras.preprocessing import image
from keras.applications.densenet import preprocess_input, decode_predictions
import numpy as np
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
import os

In [3]:
Mengumpulkan Gambar Pisang
Dataset khusus gambar pisang tanduk maupun pisang kepok agak sulit untuk ditemukan. Untungnya kita hanya memerlukan sedikit data untuk transfer learning, sehingga untuk mencari di Google Image lima puluh gambar pisang rasanya masih manusiawi. Untuk sedikit meringankan pekerjaan (kita harus memverifikasi hasil download secara manual), kita gunakan google-image-download.

Simpan gambar-gambar pisang sehingga struktur folder kita menjadi seperti berikut (andaikan kita punya 50 gambar pisang kepok dan 50 pisang tanduk.

SyntaxError: invalid syntax (<ipython-input-3-110a25d1641f>, line 1)

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory('../input/pisangtrain/pisangtrain', 
 target_size=(224,224), 
 color_mode='rgb', 
 batch_size=32, 
 class_mode='categorical', 
 shuffle=True)
category_dict = train_generator.class_indices
print(category_dict)

Found 177 images belonging to 2 classes.
{'ambon': 0, 'kepok': 1}


In [5]:
number_of_classes = len(category_dict)
base_model = densenet.DenseNet121(weights='../input/petfinder-densenet/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5',include_top=False)
#base_model =densenet.DenseNet121(weights=’imagenet',include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
preds = Dense(number_of_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)
# Print the updated layer names.
# for i,layer in enumerate(model.layers): print(i,layer.name)
# Set the first n_freeze layers of the network to be non-trainable.
n_freeze = 300
for layer in model.layers[:n_freeze]:
 layer.trainable=False
for layer in model.layers[n_freeze:]:
 layer.trainable=True

In [6]:
print(len(model.layers))

432


In [7]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
_______________________________________________________________________________________

In [8]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
step_size_train = train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator, 
 steps_per_epoch=step_size_train, 
 epochs=10)

Epoch 1/10
5/5 [==============================] - 14s 3s/step - loss: 1.1566 - accuracy: 0.4966
Epoch 2/10
5/5 [==============================] - 13s 3s/step - loss: 0.4718 - accuracy: 0.8138
Epoch 3/10
5/5 [==============================] - 12s 2s/step - loss: 0.2428 - accuracy: 0.8828
Epoch 4/10
5/5 [==============================] - 15s 3s/step - loss: 0.1612 - accuracy: 0.9310
Epoch 5/10
5/5 [==============================] - 13s 3s/step - loss: 0.1859 - accuracy: 0.9448
Epoch 6/10
5/5 [==============================] - 15s 3s/step - loss: 0.2413 - accuracy: 0.9250
Epoch 7/10
5/5 [==============================] - 14s 3s/step - loss: 0.1271 - accuracy: 0.9500
Epoch 8/10
5/5 [==============================] - 13s 3s/step - loss: 0.0817 - accuracy: 0.9586
Epoch 9/10
5/5 [==============================] - 15s 3s/step - loss: 0.0862 - accuracy: 0.9724
Epoch 10/10
5/5 [==============================] - 13s 3s/step - loss: 0.0828 - accuracy: 0.9655


In [9]:
# Without transfer learning.
#default_model = densenet.DenseNet121(weights='../input/petfinder-densenet/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5')
default_model = densenet.DenseNet121(weights='../input/petfinder-densenet/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5',include_top=False)
#densenet.DenseNet121(weights=’imagenet',include_top=True)

In [10]:
test_path = '../input/pisangtest/pisangtest/'
#test_path = 'pisang-test/'

for directory in os.listdir(test_path):
    
    # Load image.
    img_path = test_path+directory
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    default_preds = default_model.predict(x)

    # Printing results.

    # Default 1000 classes (without transfer learning).
    #print(f"Without Transfer Learning Top-2 [{directory}]: \n{decode_predictions(default_preds, top=2)[0]}\n")

    # Print transfer learning model top-1
    confidence_array = preds[0]
    index_max = np.argmax(confidence_array)

    # Get KEY (category) by VALUE (index_max) in dictionary
    # mydict = {'george':16,'amber':19}
    # print(list(mydict.keys())[list(mydict.values()).index(16)]) # Example in one line.

    category_names = category_dict.keys()
    category_values = category_dict.values()
    category_at_index = list(category_values).index(index_max)
    category_max = list(category_names)[category_at_index]

    print(f"\nWith Transfer Learning [{directory}]: \nTop-1 (confidence)\n{category_max} ({max(confidence_array)*100}%)")

    # Print transfer learning model all classes
    print("\nClass (confidence)")

    for category in category_dict:
        category_index = category_dict[category]
        value = confidence_array[category_index] * 100
        print(f"{category} ({value}%)")

    print("\n============================\n")


With Transfer Learning [kepoks (80).jpg]: 
Top-1 (confidence)
kepok (97.75749444961548%)

Class (confidence)
ambon (2.2424979135394096%)
kepok (97.75749444961548%)



With Transfer Learning [ambon99.jpg]: 
Top-1 (confidence)
kepok (99.98472929000854%)

Class (confidence)
ambon (0.01526343694422394%)
kepok (99.98472929000854%)



With Transfer Learning [kepoks (63).jpg]: 
Top-1 (confidence)
ambon (63.32191824913025%)

Class (confidence)
ambon (63.32191824913025%)
kepok (36.67807579040527%)



With Transfer Learning [ambon3.jpg]: 
Top-1 (confidence)
ambon (99.98748302459717%)

Class (confidence)
ambon (99.98748302459717%)
kepok (0.012515884009189904%)



With Transfer Learning [ambon6.jpg]: 
Top-1 (confidence)
ambon (99.75109100341797%)

Class (confidence)
ambon (99.75109100341797%)
kepok (0.24891083594411612%)



With Transfer Learning [ambon79.jpg]: 
Top-1 (confidence)
kepok (97.28819131851196%)

Class (confidence)
ambon (2.711804583668709%)
kepok (97.28819131851196%)



With Transfer

https://keras.io/api/applications/#resnet

In [11]:
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.xception import preprocess_input, decode_predictions
import numpy as np
 
import PIL
from PIL import Image
import requests
from io import BytesIO
 
 
# load the model
model = Xception(weights='imagenet', include_top=True)
 
 

img = image.load_img('../input/pisangtest/pisangtest/ambon1.jpg')
 
# resize the image according to each model (see documentation of each model)
img = img.resize((299,299))

 
##############################################
# if you want to read the image from your PC 
#############################################
# img_path = 'myimage.jpg'
# img = image.load_img(img_path, target_size=(299, 299))
#############################################
 
 
 
# convert to numpy array
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
 
features = model.predict(x)
 
# return the top 20 detected objects
label = decode_predictions(features, top=10)
label

40960/35363 [==================================] - 0s 0us/step


[[('n07753592', 'banana', 0.6963459),
  ('n07716358', 'zucchini', 0.012368389),
  ('n03461385', 'grocery_store', 0.011901178),
  ('n07717556', 'butternut_squash', 0.0073917033),
  ('n02927161', 'butcher_shop', 0.002438676),
  ('n07718472', 'cucumber', 0.0024377808),
  ('n13133613', 'ear', 0.0020470473),
  ('n07747607', 'orange', 0.002003146),
  ('n07716906', 'spaghetti_squash', 0.0019953689),
  ('n03127925', 'crate', 0.0019100723)]]